In [3]:
#Libraries
import pandas as pd
import re
import requests

import os
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import config 

from getpass import getpass

import random

from selenium.webdriver.common.action_chains import ActionChains

In [4]:
## first we need to initiate the driver##
driver = webdriver.Chrome('C:\\Users\\catar\\OneDrive\\Documentos\\chromedriver.exe')

In [5]:
# open the website
driver.get('https://www.glassdoor.com')

In [6]:
# Login in
signin = driver.find_elements_by_css_selector("button[class^='d-flex align-items-center justify-content-center order-1 order-md-2 mr-auto mr-md-0 p-0 LockedHomeHeaderStyles__signInButton']")[0].click()

time.sleep(2.5) ## sleep timer in order to mimic human interaction

In [5]:
# Insert email
email = getpass('Enter your email') ## input email

email_box = driver.find_element_by_id("userEmail") ## box to input email

email_box.send_keys(email)  ## send email to webpage

time.sleep(3.5) ## sleep timer in order to mimic human interaction

In [6]:
#Insert password

password = getpass('Enter your password') ## input password

pass_box = driver.find_element_by_id('userPassword') ## box to input password

pass_box.send_keys(password)  ## send password to webpage

time.sleep(1.3) ## sleep timer in order to mimic human interaction

In [7]:
#login

login = driver.find_elements_by_css_selector("button[class^='gd-ui-button minWidthBtn css-8i7bc2']")[0].click()

time.sleep(1.6)  ## sleep timer in order to mimic human interaction


In [8]:
#######################################

In [8]:
# Lets try a function for that

def interview_info(job_title,location):
    
    actions = ActionChains(driver)
    
    # first lets go to interview page
    interview_icon = "//a[@data-test = 'site-header-interviews']" 
    interview_button = driver.find_element_by_xpath(interview_icon)
    interview_click = interview_button.click()
    time.sleep(random.uniform(5,7)) ## sleep timer in order to mimic human interaction
    
    # lets input the job title to search
    job_title_box = driver.find_element_by_id("KeywordSearch") ## box to input job title
    job_title_box.send_keys(job_title) ## send job title to page
    time.sleep(random.uniform(5,7)) ## sleep timer in order to mimic human interaction
    
    # lets input the location to search
    location_box = driver.find_element_by_id("LocationSearch")  ## box to input job title
    location_box.clear() ##clear initial input
    location_box.send_keys(location) ## send job title to page
    time.sleep(random.uniform(5,7)) ## sleep timer in order to mimic human interaction
    
    # find information
    interview_search = driver.find_element_by_id('HeroSearchButton') ## click search button
    interview_search.click()  ## click search button
    time.sleep(random.uniform(5,7))  ## sleep timer in order to mimic human interaction
    
    ## Now that we have the results lets check total number of results and page of results
    results = "//div[@class='reviewCount center']"
    total_results = driver.find_element_by_xpath(results).text
    number_of_results = int(total_results)
    number_of_pages = round(number_of_results/10)

    ## Create to add the info scrapped
    job_company = []
    questions = []
    answers = []

    ## Actions
    actions = ActionChains(driver)
    
    #lets iterate throw the questions
    for i in range(1,(number_of_results+1)):
        display(i)
        
        #company info
        xpath1 = "//div[starts-with(@id, 'InterviewQuestionResult_"+str(i)+"')]//span[@class='authorInfo']"
        job_company_name = driver.find_element_by_xpath(xpath1)
        job_company_name_text = job_company_name.text
        display(job_company_name_text)
        job_company.append(job_company_name_text)
        time.sleep(random.uniform(3,6))

        #question info
        xpath2 = "//div[starts-with(@id, 'InterviewQuestionResult_"+str(i)+"')]//p[@class='questionText h3']"
        questions_gd = driver.find_element_by_xpath(xpath2)
        questions_gd_text = questions_gd.text
        display(questions_gd_text)
        questions.append(questions_gd_text)
        time.sleep(random.uniform(3,6))

        #answer info
        try:
            #click answer button
            if ((i%10) != 1):   
                xpath3 = "//div[starts-with(@id, 'InterviewQuestionResult_"+str(i)+"')]//a[@class='userResponseLink margTop block hiddenLink mmLink ']"
                answer_icon = driver.find_element_by_xpath(xpath3)
                answer_icon_click = answer_icon.click()
                time.sleep(random.uniform(3,6))         
        
            #get answer info            
            xpath4 = "//div[starts-with(@id, 'InterviewQuestionResult_"+str(i)+"')]//div[@class='responseText padTopSm tbl fill']"
            answers_gd = driver.find_element_by_xpath(xpath4)
            answers_gd_text = answers_gd.text
            display(answers_gd_text)
            answers.append(answers_gd_text)
            time.sleep(random.uniform(3,6))
        
        #if no answer presented
        except:
            answers_gd_text = 'No answer'
            display(answers_gd_text)
            answers.append(answers_gd_text)
            time.sleep(random.uniform(3,6))
            
        #go to next page
        if ((i%10) == 0):
            xpath_page = "//div[@class='pagingControls cell middle']//li[@class ='next']"
            page_change = driver.find_element_by_xpath(xpath_page)
            page_change_button = page_change.click()
            time.sleep(random.uniform(15,20))
    
    
    #lets retrieve dataframe with this info
    interview_data = pd.DataFrame([job_company,questions,answers]).T
    interview_data.columns = ['job_company', 'questions','answers']
    interview_data['country'] = location
    interview_data['area'] = job_title
    display(interview_data)

    return interview_data

In [9]:
def data_to_csv(location,scrap_data):
    t = time.localtime() # check time
    m = str(t.tm_mon) #scrap month
    d = str(t.tm_mday) #scrap day
    h = str(t.tm_hour) #scrap hour
    mt = str(t.tm_min) #scrap minute

    #lets structure how to present this info in the file name
    structure = str(location_pt)+"_m"+str(m)+"_d"+str(d)+"_h"+str(h)+"_"+str(mt)

    #turn into csv
    scrap_data.to_csv("interview_data_"+structure+".csv",sep = ',',index=False)


In [22]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'Portugal' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Senior Data Scientist at OLX Group was asked...'

'explain the reasoning in resolving the case'

'I answer all the insights I had based on data analysis and business acumen'

2

...

66

'Junior Data Scientist at FARFETCH was asked...'

"They asked me to explain the advantages of my case-study solution to a business person, who don't know anything about to Data Science and Machine Learning."

'No answer'

,job_company,questions,answers,country,area
0,Senior Data Scientist at OLX Group was asked...,explain the reasoning in resolving the case,I answer all the insights I had based on data ...,Portugal,Data Analyst
1,Data Science at MOG Technologies was asked...,"Nothing too technical, they presented one of t...",I told them that it could be solved and presen...,Portugal,Data Analyst
2,Data Scientist at Smart Consulting was asked...,Tell me about 2 projects that you seem fit to ...,Described two projetcs about AI,Portugal,Data Analyst
3,Data Scientist at Deloitte was asked...,What your motivations are?,"If the data adequately represent reality, by s...",Portugal,Data Analyst
4,Data Scientist at 360imprimir was asked...,Logic question( cases resolution),Detailed explanation about the reasoning behin...,Portugal,Data Analyst
...,...,...,...,...,...
61,Data Scientist at Builtrix was asked...,Questions related to your past work experience...,No answer,Portugal,Data Analyst
62,Data Scientist at Feedzai was asked...,"that is quite fuzzy screening test, with neces...",No answer,Portugal,Data Analyst
63,Data Scientist at Feedzai was asked...,Describe two of your past projects that had Ma...,No answer,Portugal,Data Analyst
64,Data Scientist at Miniclip was asked...,- Technical questions about the challenge,No answer,Portugal,Data Analyst


In [24]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'Portugal' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Web Developer Junior at Prime IT was asked...'

'Seu eu sabia o que fazia a Prime IT'

'Falando o que havia lido a respeito da empresa e de seus metodos de trabalho'

2

...

31

'Full Stack Web Developer at ITSector was asked...'

'On a blank board do a simple algorithm to find a capicua.'

'No answer'

,job_company,questions,answers,country,area
0,Web Developer Junior at Prime IT was asked...,Seu eu sabia o que fazia a Prime IT,Falando o que havia lido a respeito da empresa...,Portugal,Web Developer
1,Web Developer at Leya was asked...,They asked all the usual job interview questio...,"I wrote down the general programming logic, in...",Portugal,Web Developer
2,Web Developer at Landing.jobs was asked...,If I would feel Ok with unpaid extra time ever...,Obviously not.,Portugal,Web Developer
3,Junior Web Developer at iTGROW was asked...,Difference between foreach and map Java functions,"Map returns values, foreach doesn't,...",Portugal,Web Developer
4,Web Developer at PDM&FC was asked...,I was asked about Java and JavaScript even tho...,No answer,Portugal,Web Developer
5,Web Developer at Wiremaze was asked...,What's your salary expectation?,No answer,Portugal,Web Developer
6,Web Developer at Eticadata was asked...,"Database questions, CRUD",No answer,Portugal,Web Developer
7,Web Developer at rVisual was asked...,they asked me to fix old broken code without b...,No answer,Portugal,Web Developer
8,Web Developer at Wiremaze was asked...,"I was asked to do a simple HTML page, with som...",No answer,Portugal,Web Developer
9,Web Developer at Alter Solutions was asked...,What kind of projects that you work?,No answer,Portugal,Web Developer


In [25]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'Portugal' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UX Designer at Sport Zone was asked...'

'Can you work under pressure?'

'Yes.'

2

...

11

'UI/UX Designer at OutSystems was asked...'

'Describe the UX process you are aware of.'

'No answer'

,job_company,questions,answers,country,area
0,UX Designer at Sport Zone was asked...,Can you work under pressure?,Yes.,Portugal,UX Designer
1,UX Designer at Sport Zone was asked...,Why do you want to leave you actual job?,I want a new experience.,Portugal,UX Designer
2,UX Designer at Sport Zone was asked...,Talk about you and your experience.,I just focused on my experience.,Portugal,UX Designer
3,UI & UX Designer at BNP Paribas was asked...,Previous experiences with data visualization,No answer,Portugal,UX Designer
4,UX Designer at Zalando was asked...,"The interview was more about my experience, me...",No answer,Portugal,UX Designer
5,UI/UX Designer at Pixelmatters was asked...,What is for you/your understading of UI/UX Des...,No answer,Portugal,UX Designer
6,Ui\Ux Designer at YGroup Companies was asked...,"If you had to validate this solution, how woul...",No answer,Portugal,UX Designer
7,UX Designer at Sport Zone was asked...,What do you expect from this job?,Be happy doing what I love to do. A fair salar...,Portugal,UX Designer
8,UX Designer at Feedzai was asked...,They questioned Front-End Development instead ...,No answer,Portugal,UX Designer
9,Sr UX Designer 2 at Springer Nature was asked...,"Tell me about your experience, explain your de...",No answer,Portugal,UX Designer


In [26]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'Portugal' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UI Designer at WIT Software was asked...'

'Why do you want to join us? What motivates you?'

'To live and learn, to contribute and to evolve, because what motivates me to do this, my children.'

2

...

8

'UI/UX Designer at OutSystems was asked...'

'Describe the UX process you are aware of.'

'No answer'

,job_company,questions,answers,country,area
0,UI Designer at WIT Software was asked...,Why do you want to join us? What motivates you?,"To live and learn, to contribute and to evolve...",Portugal,UI Designer
1,UX-UI Designer at NDrive was asked...,- Why were i interested in working there and h...,No answer,Portugal,UI Designer
2,UI & UX Designer at BNP Paribas was asked...,Previous experiences with data visualization,No answer,Portugal,UI Designer
3,UI/UX Designer at Pixelmatters was asked...,What is for you/your understading of UI/UX Des...,No answer,Portugal,UI Designer
4,Ui\Ux Designer at YGroup Companies was asked...,"If you had to validate this solution, how woul...",No answer,Portugal,UI Designer
5,UX-UI Designer at BOOST IT was asked...,The interview was mostly a Q&A session based o...,No answer,Portugal,UI Designer
6,User Experience/UI Designer at Blip was asked...,"Regular questions about your method, collabora...",No answer,Portugal,UI Designer
7,UI/UX Designer at OutSystems was asked...,Describe the UX process you are aware of.,No answer,Portugal,UI Designer


In [17]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'Spain' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Data Analyst at Electronic Arts was asked...'

'What was one of the biggest challenge on your last work? Are you more an analytical or creative person?'

'I answered with a challenge I had when I had to start a unit from scratch.'

2

...

181

'Data Scientist at TeamCMP was asked...'

'Describe como funciona un algoritmo supervisado de ML. Describe como funciona un no supervisado. Preguntas de experiencia y como lo resolviste.'

'No answer'

,job_company,questions,answers,country
0,Data Analyst at Electronic Arts was asked...,What was one of the biggest challenge on your ...,I answered with a challenge I had when I had t...,Spain
1,Business Intelligence and Data Science Analyst...,How many universes did I managed,Really I have enough with mine ;),Spain
2,Data Scientist at Typeform was asked...,How experienced are you with SQL?,Sounds about right -- shady company.,Spain
3,Data Scientist at King was asked...,How would you implement a factorial function w...,Recursion function factorial(n) { if n = 0 ret...,Spain
4,Data Analyst/Data Scientist at Boston Consulti...,Specific bussiness related questions to a case...,Calculations. You only have 1:30 seconds per q...,Spain
...,...,...,...,...
176,Data Analyst at Morningstar was asked...,Why do you think yourself best candidate for t...,No answer,Spain
177,Data Scientist at TeamCMP was asked...,Describe como funciona un algoritmo supervisad...,No answer,Spain
178,Data Scientist at Criteo was asked...,Why Criteo? What makes Criteo different from t...,No answer,Spain
179,Analyst Data Science at Criteo was asked...,How would you explain Criteo's business model ...,No answer,Spain


In [27]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'Spain' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Web Developer at Glovo was asked...'

'"Imagine its your first week in the company and I ask you to do a presentation of your choice in front of your colleagues, what would it be about?"'

'My Chioce will be service render'

2

...

22

'.NET Web Developer at Electronic Arts was asked...'

'Familiar with Agile?'

'No answer'

,job_company,questions,answers,country,area
0,Web Developer at Glovo was asked...,"""Imagine its your first week in the company an...",My Chioce will be service render,Spain,Web Developer
1,Web Developer at Glovo was asked...,They asked me to explain how the Internet works,First your network must be strong and fast you...,Spain,Web Developer
2,Web Developer at Adaptive Financial Consulting...,Tell us about component lifecycle in React.,"Mentioned the most useful core methods, but th...",Spain,Web Developer
3,Junior Web Developer at Marfeel was asked...,"You don't have to say if you don't want to, bu...",I told them the range of how much it was.,Spain,Web Developer
4,Web Developer at Avature was asked...,Revisión de código + 2 ejercicios de programac...,No answer,Spain,Web Developer
5,Desarrollador Web at Denodo was asked...,Unas 30 preguntas técnicas tipo test que la ma...,No answer,Spain,Web Developer
6,Junior Web Developer at Netguru was asked...,They gave me a practical task to complete.,No answer,Spain,Web Developer
7,Desarrollador Web Junior at Sopra Steria was a...,Acerca de mis intereses e inquietudes,No answer,Spain,Web Developer
8,Full Stack Web Developer at Gamesys was asked...,Much more...,No answer,Spain,Web Developer
9,Full Stack Web Developer at Gamesys was asked...,what's immutability? explain,No answer,Spain,Web Developer


In [28]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'Spain' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UX Designer at Hostaway was asked...'

'How will you find people for usability testing if you cannot find any existing clients locally?'

'Usability testing is better to be done with people who have not interacted with product before. So, we are escaping the biased answers and personal likes/dislikes and evaluating the usability of a system from an independent point of view.'

2

...

40

'UI/UX Designer at Kantox was asked...'

'A superficial usability analysis of their existing product.'

'No answer'

,job_company,questions,answers,country,area
0,UX Designer at Hostaway was asked...,How will you find people for usability testing...,Usability testing is better to be done with pe...,Spain,UX Designer
1,UI/UX Designer at Seocom was asked...,Since I dont know spanish he asked how much i ...,I said no but that I am a fast learner and I u...,Spain,UX Designer
2,UX Designer at GFT Technologies was asked...,They did asked difficult questions,Nothing was unexpected,Spain,UX Designer
3,UI-UX Designer at Hewlett Packard Enterprise |...,What were my current responsibilities,Saying what I do actually execute in my curren...,Spain,UX Designer
4,UX Designer at Wysiwyg was asked...,How do you propose a user test?,No answer,Spain,UX Designer
5,Diseñador Ux/Ui at KingEclient was asked...,Parece que antes de se entrevistado como tal y...,"Apenas hablé 3 minutos con la seleccionadora, ...",Spain,UX Designer
6,UX Designer at Domestika was asked...,Experiencia pasada.,No answer,Spain,UX Designer
7,UX Designer at New Work SE was asked...,How is your current team set up?,No answer,Spain,UX Designer
8,Senior UX Designer at King was asked...,Most questions were related to personal work e...,No answer,Spain,UX Designer
9,UI/UX Designer at TeamCMP was asked...,What would be your ideal working environment?,No answer,Spain,UX Designer


In [29]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'Spain' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UX/UI Designer at Open Assessment Technologies was asked...'

'Are you able to sign an agreement with a university or a business school or any other institution?'

'No, I am not.'

2

...

41

'UI/UX Designer at Kantox was asked...'

'A superficial usability analysis of their existing product.'

'No answer'

,job_company,questions,answers,country,area
0,UX/UI Designer at Open Assessment Technologies...,Are you able to sign an agreement with a unive...,"No, I am not.",Spain,UI Designer
1,UI Designer at netzstrategen was asked...,Why do you want to change your current job.,Because I was looking for a different work cul...,Spain,UI Designer
2,Graphic/UX-UI Designer at payever was asked...,Are you okay with 10-20 usd/hour?,Are you okay working weekends?,Spain,UI Designer
3,UI/UX Designer at Seocom was asked...,Since I dont know spanish he asked how much i ...,I said no but that I am a fast learner and I u...,Spain,UI Designer
4,UI-UX Designer at Hewlett Packard Enterprise |...,What were my current responsibilities,Saying what I do actually execute in my curren...,Spain,UI Designer
5,Junior User Experience UI Designer at KingEcli...,"Personal questions, nothing unexpected.",Just being myself.,Spain,UI Designer
6,Senior UX/UI Designer at Lookiero was asked...,"Cuál ha sido mi trayectoria profesional, estud...","Con sinceridad, contando los hitos, problemas,...",Spain,UI Designer
7,UI/User Experience Designer at Ubeeqo was aske...,The questions were quite normal,No answer,Spain,UI Designer
8,Diseñador Ux/Ui at KingEclient was asked...,Parece que antes de se entrevistado como tal y...,"Apenas hablé 3 minutos con la seleccionadora, ...",Spain,UI Designer
9,UX-UI Designer at ALUXION LABS was asked...,Preguntas relacionadas al proceso de diseño + ...,No answer,Spain,UI Designer


In [35]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'Germany' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Data Scientist at Kreditech Holding was asked...'

'Present a previous project and they will ask a lot of technical questions regarding the topic.'

'Hello, can you give the details of the task that was given in the test.'

2

...

451

'Data Scientist at Funding Circle was asked...'

'Generic questions, tell me about yourself? Why this job? Why Funding Circle?'

'No answer'

,job_company,questions,answers,country,area
0,Data Scientist at Kreditech Holding was asked...,Present a previous project and they will ask a...,"Hello, can you give the details of the task th...",Germany,Data Analyst
1,Data Analyst at Zalando was asked...,"- SQL: they use PostgreSQL, asked something re...",be nice and have a nice conversation with them...,Germany,Data Analyst
2,Data Analyst at Zalando was asked...,Why do you want to join zalando?,I did not,Germany,Data Analyst
3,Research Engineer/Data Science at Zalando was ...,Mostly technical / coding questions with limit...,Could you provide some examples of the coding ...,Germany,Data Analyst
4,Senior Data Analyst at Chrono24 was asked...,Warum will ich meinen derzeitigen Arbeitgeber ...,Meine Situation geschildert,Germany,Data Analyst
...,...,...,...,...,...
446,Data Analyst at Accenture was asked...,Warum Accenture?,No answer,Germany,Data Analyst
447,Junior Data Scientist at TeamViewer was asked...,2 simple SQL questions were asked,No answer,Germany,Data Analyst
448,Data Scientist at Funding Circle was asked...,"Generic questions, tell me about yourself? Why...",No answer,Germany,Data Analyst
449,Junior Data Scientist at Deloitte was asked...,Was ist der Unterschied zwischen Klassifizieru...,No answer,Germany,Data Analyst


In [32]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'Germany' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Web Developer at NavVis was asked...'

'This greatly depends on which role you are interviewing for'

'Mechanical engineering'

2

...

66

'Part-Time Web Developer at Kolibri Games was asked...'

'What makes a good team / a good working experience?'

'No answer'

,job_company,questions,answers,country,area
0,Web Developer at NavVis was asked...,This greatly depends on which role you are int...,Mechanical engineering,Germany,Web Developer
1,Front End Web Developer at Eurowings Digital w...,Coding challenge was to integrate the Imgur AP...,"I created the application in Angular 8, along ...",Germany,Web Developer
2,Web Developer at payever was asked...,They will provide you a task and then judge yo...,"Their task setup was incomplete, very poorly c...",Germany,Web Developer
3,Web Developer at smava was asked...,Q: Experience with React? Q: Experience with s...,A: List of previous jobs with related tech.,Germany,Web Developer
4,Frontend Web Developer at Delivery Hero was as...,Js tasks,"4/5, make some errors",Germany,Web Developer
...,...,...,...,...,...
61,Senior Web Developer at Pepper was asked...,"What SOLID stands for, how would you describe ...",No answer,Germany,Web Developer
62,Full Stack Web Developer at innoscripta was as...,one of the algorithmic problems was related to...,No answer,Germany,Web Developer
63,Senior Web Developer at Zalando was asked...,Why do I want to work for them?,No answer,Germany,Web Developer
64,Web Developer at Westwing Home and Living was ...,How many squares on a chess board?,No answer,Germany,Web Developer


In [33]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'Germany' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UX Designer at USEEDS° was asked...'

'warum useeds'

'bisheriges lieblingsprojekt'

2

...

72

'UI-UX Designer at Wunderflats was asked...'

'What is your design process and how do you solve design problems?'

'No answer'

,job_company,questions,answers,country,area
0,UX Designer at USEEDS° was asked...,warum useeds,bisheriges lieblingsprojekt,Germany,UX Designer
1,UX Designer at Cornelsen Schulverlage was aske...,What does a typically UX proces look like for ...,I normally start out setting visions and goals...,Germany,UX Designer
2,UX Designer at Homebell was asked...,How would you change this specific design of t...,I showed a wireframe but they didnt like it to...,Germany,UX Designer
3,UX Designer at Zoi was asked...,Endlose Ausführungen über die Firmengeschichte...,Wenn ich die Website nicht selbst lesen könnte...,Germany,UX Designer
4,UI-UX Designer at REBELLE - StyleRemains was a...,What were your UI-UX responsibilities in your ...,Design and usability of the Website......,Germany,UX Designer
...,...,...,...,...,...
67,Senior UX Designer at trivago was asked...,They aked about my case study. Why I made the ...,No answer,Germany,UX Designer
68,UI/UX Designer at Global Savings Group was ask...,Why did you apply for this position?,No answer,Germany,UX Designer
69,UI-UX Designer at Gymondo was asked...,What's your role in your current team?,No answer,Germany,UX Designer
70,Senior UX Designer at Quandoo was asked...,"Generally, the questions were fairly straight-...",No answer,Germany,UX Designer


In [34]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'Germany' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UI Designer at scoutbee was asked...'

'Could you describe your typical design process?'

'I gave an in-depth walk-through of my typical process, and gave examples from my specific experiences, highlighting what I can carry over to benefit the company'

2

...

84

'UI Designer at Kolibri Games was asked...'

'Superficial Questions about CV'

'No answer'

,job_company,questions,answers,country,area
0,UI Designer at scoutbee was asked...,Could you describe your typical design process?,I gave an in-depth walk-through of my typical ...,Germany,UI Designer
1,UI-UX Designer at REBELLE - StyleRemains was a...,What were your UI-UX responsibilities in your ...,Design and usability of the Website......,Germany,UI Designer
2,UI-UX Designer at Native Instruments was asked...,What do you expect from this job?,Designing interfaces and workflows.,Germany,UI Designer
3,Senior UI/User Experience Designer at Native I...,I've applied to a Senior UX/UI Designer positi...,"I've answered that, since I had applied to a U...",Germany,UI Designer
4,UI-UX Designer at Solvemate was asked...,Working for free on unpaid offsite tasks,No,Germany,UI Designer
...,...,...,...,...,...
79,UI-UX Designer at Gymondo was asked...,What's your role in your current team?,No answer,Germany,UI Designer
80,UI-UX Designer at Gymondo was asked...,What's your role in your current team?,No answer,Germany,UI Designer
81,UI-UX Designer at Wunderflats was asked...,What is your design process and how do you sol...,No answer,Germany,UI Designer
82,UI Designer at Quandoo was asked...,A few simple questions,No answer,Germany,UI Designer


In [11]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'Italy' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UI/UX Designer at Bit4id was asked...'

"Come mai volevo lavorare all'interno dell'azienda."

'La reputo nello scenario globale una delle aziende meglio strutturate e solide.'

2

...

17

'UX Designer at Bending Spoons was asked...'

'How did you get into UX? And how did your vision of it change in the past year.'

'No answer'

,job_company,questions,answers,country,area
0,UI/UX Designer at Bit4id was asked...,Come mai volevo lavorare all'interno dell'azie...,La reputo nello scenario globale una delle azi...,Italy,UX Designer
1,Senior UX Designer at Bending Spoons was asked...,I was asked to spot bad UX in provided wirefra...,tried to be as clear as possible but the time ...,Italy,UX Designer
2,UI/UX Designer at Oval Money was asked...,Complete the task included in the pdf.,We are sorry that you perceive you had a negat...,Italy,UX Designer
3,Junior UX Designer at DOING was asked...,Mi è piaciuto l'interesse mostrato per il mio ...,Ho raccontato la metodologia seguita e i tool ...,Italy,UX Designer
4,UI & UX Designer at Apple Developer Academy wa...,Would you be willing to live in Naples?,"Tell me about yourself, how do you keep inform...",Italy,UX Designer
5,UX Designer at Bending Spoons was asked...,Combination of long response form and multiple...,In respect for the process I won't elaborate. ...,Italy,UX Designer
6,UX Designer at Prima Assicurazioni was asked...,Come ti tieni aggiornata?,No answer,Italy,UX Designer
7,UI/UX Designer at Docebo was asked...,Quali sono state le mie ultime esperienze lavo...,No answer,Italy,UX Designer
8,UX Designer at Bending Spoons was asked...,How would you improved this ux journey you are...,No answer,Italy,UX Designer
9,Junior UX Designer at Conflux Technologies was...,Parlaci degli ultimi progetti che hai fatto.,No answer,Italy,UX Designer


In [12]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'Sweden' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UX Designer at Consafe Logistics Group was asked...'

'About my work , experience and projects'

'I was consisted in answers .'

2

...

13

'UX Designer at Cambio Healthcare Systems was asked...'

'How would you cooperate with marketing department'

'No answer'

,job_company,questions,answers,country,area
0,UX Designer at Consafe Logistics Group was ask...,"About my work , experience and projects",I was consisted in answers .,Sweden,UX Designer
1,UX Designer at One Planet Rating was asked...,Reactjs and old projects?,the basic flow of the react,Sweden,UX Designer
2,UX Designer at One Planet Rating was asked...,Javascript latest update?,Es8 is the latest update.,Sweden,UX Designer
3,UX Designer at H&M was asked...,How might you improve this project?,Add rapid user test after prototyping and visu...,Sweden,UX Designer
4,UX Designer at Bontouch was asked...,Describe your design process.,No answer,Sweden,UX Designer
5,UX Designer at iZettle was asked...,What is your design process? How do you conduc...,No answer,Sweden,UX Designer
6,UX Designer at King was asked...,Why do you want to work on king,No answer,Sweden,UX Designer
7,UX Designer at H&M was asked...,What sets you apart from other designers?,No answer,Sweden,UX Designer
8,UX Designer at Cybercom Group was asked...,Have you done A/B testing?,No answer,Sweden,UX Designer
9,UX Designer at ASSA ABLOY was asked...,How would you deal with a team member that doe...,No answer,Sweden,UX Designer


In [13]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'United Kingdom' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UI-UX Designer at SendBird was asked...'

'The company does not interview candidates, only receiving résumés and portfolios.'

'I am sorry you had a negative experience. Unfortunately, we do not have any UX roles open in UK, nor have we ever had a role posted in that location or have any plans to hire for that role in UK/EMEA. Our UX team is located in Seoul Korea.'

2

...

93

'UX Designer at ThoughtWorks was asked...'

'What kind of experience have you had?'

'No answer'

,job_company,questions,answers,country,area
0,UI-UX Designer at SendBird was asked...,"The company does not interview candidates, onl...",I am sorry you had a negative experience. Unfo...,United Kingdom,UX Designer
1,UI-UX Designer at Firefly (London) was asked...,Tell us about the project you're most proud of.,How do you work with developers?,United Kingdom,UX Designer
2,UI-UX Designer at LateRooms.com was asked...,How would you resolve conflicting stakeholder ...,prototype them and test them and let the user ...,United Kingdom,UX Designer
3,UI-UX Designer at Mindtree was asked...,they asked questions related to my presentation,simply replied to that,United Kingdom,UX Designer
4,UX Designer at elder was asked...,Go through a recent project I have worked on?,"I went over a recent project, but was not aske...",United Kingdom,UX Designer
...,...,...,...,...,...
88,UX Designer at Microlise was asked...,Describe what User Experience means to you?,No answer,United Kingdom,UX Designer
89,Senior UX Designer at Amazon was asked...,Describe a time where you made a mistake on a ...,No answer,United Kingdom,UX Designer
90,UX Designer at OVO was asked...,General questions regarding experience and ski...,No answer,United Kingdom,UX Designer
91,UX Designer at Rapid7 was asked...,General questions based around the topic of re...,No answer,United Kingdom,UX Designer


In [14]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'Brazil' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UI/UX Designer Sênior at Agência Bend was asked...'

'Sobre inteligência emocional, pois hoje é muito importante a comunicação entre a equipe.'

'Sou uma pessoa comunicativa e tento sempre manter a tranparência, motivaçnao e a equipe unida.'

2

...

203

'UI/UX Designer at XYZ (Hong Kong) was asked...'

'Pretensão salarial, contar um pouco sobre minha experiência experiencia profissional, se tenho filhos etc'

'No answer'

,job_company,questions,answers,country,area
0,UI/UX Designer Sênior at Agência Bend was aske...,"Sobre inteligência emocional, pois hoje é muit...",Sou uma pessoa comunicativa e tento sempre man...,Brazil,UX Designer
1,UI/UX Designer at Instituto de Desenvolvimento...,O fato de você ser muito centrado te atrapalha...,"Que atrapalhar, atrapalha. Mas não significati...",Brazil,UX Designer
2,UX Designer at MobLee was asked...,Se interessaram pelo processo de criação usand...,Desenvolvi a partir do que havia feito.,Brazil,UX Designer
3,UI/UX Designer Estágio at Reino Studio was ask...,qual seu processo de produção de um app?,"pesquisa, fluxograma, wireframe, entrevista co...",Brazil,UX Designer
4,UI/UX Designer at Revelo was asked...,Perguntas sobre marketing como Roi de campanha.,Minha area é de design,Brazil,UX Designer
...,...,...,...,...,...
198,UX Designer Pleno at CI&T was asked...,Como você acha que a CI&T pode ajudar na sua c...,No answer,Brazil,UX Designer
199,UI/UX Designer Pleno at CI&T was asked...,Foi mais uma conversa aberta e fluída que perg...,No answer,Brazil,UX Designer
200,UX Designer at Grupo FCamara was asked...,Qual seu projeto de vida?,No answer,Brazil,UX Designer
201,UI/UX Designer Júnior at TOTVS was asked...,"Perguntas específicas da área, sobre o process...",No answer,Brazil,UX Designer


In [15]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'Netherlands' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UX Designer at Bending Spoons was asked...'

"What made you interesting in applying for this position? What kind of task do you think you'd be responsible for?"

'I love to design for people, not clients. My tasks will be user/information research, conducting brainstorm sessions, conducting usability testing, creating prototypes'

2

...

38

'UI-UX Designer at albelli was asked...'

'They asked about my previous experience and preferences as a designer'

'No answer'

,job_company,questions,answers,country,area
0,UX Designer at Bending Spoons was asked...,What made you interesting in applying for this...,"I love to design for people, not clients. My t...",Netherlands,UX Designer
1,UX Designer at Booking.com was asked...,Code a post-card using html css in 30 mins,"The test is quite easy, just the time is short",Netherlands,UX Designer
2,UX Designer at DSW Zorgverzekeraar was asked...,Wat zijn je cijfers van de middelbare school?,Deze had ik toen niet parraat.,Netherlands,UX Designer
3,UX Designer at Bending Spoons was asked...,A colleague of yours talks to you privately an...,You will get a list of statements and I chose ...,Netherlands,UX Designer
4,UX Designer at Bending Spoons was asked...,Which of the following statements best describ...,You will get different statements to choose fr...,Netherlands,UX Designer
5,Senior UX Designer at albelli was asked...,Kan je met behulp van je portfolio uitleggen w...,"Ja, dat kan.",Netherlands,UX Designer
6,UX Designer at Booking.com was asked...,Do you know HTML/CSS?,Yes cause I am UI developer &amp; UX designer.,Netherlands,UX Designer
7,UI/UX Designer at Boeing was asked...,How do you manage stress at work? Share a case...,Explaining a real situation that happened to m...,Netherlands,UX Designer
8,Ui\Ux Designer at AirBeamTV was asked...,How I approach critical cases?,I explained with an example from my portfolio,Netherlands,UX Designer
9,UX Designer at Ivanti was asked...,Vind je het een probleem als een ontwerp een d...,Wauw..,Netherlands,UX Designer


In [16]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'France' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UX Designer at Synchrone was asked...'

'Décrivez votre méthodologie de travail'

'Description du workflow'

2

...

70

'UX Designer at Niji was asked...'

'Décrivez moi votre parcours Pro'

'No answer'

,job_company,questions,answers,country,area
0,UX Designer at Synchrone was asked...,Décrivez votre méthodologie de travail,Description du workflow,France,UX Designer
1,UX Designer at Nexton Consulting was asked...,"Presente toi, Quel est ton parcours ? Connais ...",Je suis mitigé par cette entreprise car j'ai e...,France,UX Designer
2,UI-UX Designer at iAdvize was asked...,Revisiter le formulaire de création d’une règl...,Maquettes graphiques + argumentaires,France,UX Designer
3,UI-UX Designer at ModuloTech was asked...,qu'est ce qu'il te saute au yeux quand tu rega...,certaines icones ui sont incohérente les une a...,France,UX Designer
4,Alternance PO - UX Designer at Allianz was ask...,Des exemples de projets que tu as mené ?,Description de mes expériences les plus marqua...,France,UX Designer
...,...,...,...,...,...
65,Junior Consultant UX Designer at mc2i was aske...,Proposez une démarche pour la conception d'une...,No answer,France,UX Designer
66,UI-UX Designer at Fifty-Five was asked...,Quelles sont vos pratiques courantes ?,No answer,France,UX Designer
67,UX Designer at SeLoger was asked...,l'intervieweur demande de faire un exercice su...,No answer,France,UX Designer
68,Senior UX Designer at BlaBlaCar was asked...,Design Exercise: Teach a user how to complete ...,No answer,France,UX Designer


In [17]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'Mexico' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UX Designer at Totalplay Empresarial was asked...'

'Experiencia en el puesto solicitado, platicar como resolví una problemática en un empleo anterior, etc y la que no me gustó fue que si tenía hijos o pienso tenerlos'

'Muy diplomática para no perder la oportunidad de trabajar ahí, pero de todos modos no me dieron el empleo'

2

...

9

'UX Designer at Rappi was asked...'

'Que tipo de metodologias usas?'

'No answer'

,job_company,questions,answers,country,area
0,UX Designer at Totalplay Empresarial was asked...,"Experiencia en el puesto solicitado, platicar ...",Muy diplomática para no perder la oportunidad ...,Mexico,UX Designer
1,Senior UX Designer at Backbase was asked...,Which project left me more lessons learned?,A project where I had to work seamlessly with ...,Mexico,UX Designer
2,Senior UX Designer at Televisa was asked...,What is the most important thing to look at in...,The end user is the key to develop a solution ...,Mexico,UX Designer
3,UX Designer at Juan Osorio López Autos was ask...,Las desventajas de trabajar con este empleador...,Las desventajas de trabajar con este empleador...,Mexico,UX Designer
4,UX Designer at Gesfor Mexico was asked...,Describir los pasos y procesos de las metodolo...,Describí el proceso de scrum y design thinking,Mexico,UX Designer
5,UI/UX Designer at EasyBroker was asked...,Examples of team work.,No answer,Mexico,UX Designer
6,Senior UX Designer at eDataWorld was asked...,- if I could live in US? - if I was single? - ...,No answer,Mexico,UX Designer
7,UX Designer at Investimento was asked...,Puedes trabajar en equipos multidisciplinarios...,No answer,Mexico,UX Designer
8,UX Designer at Rappi was asked...,Que tipo de metodologias usas?,No answer,Mexico,UX Designer


In [18]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'Mexico' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Analista De Base De Datos at Posadas was asked...'

'¿Que posición tomarías en caso de que te enteres que un compañero de trabajo esta haciendo mal uso de la información de la compañía?'

'Denunciar inmediatamente el mal uso de dicha información.'

2

...

39

'Analista De Base De Datos at PepsiCo was asked...'

'Que nivel de SQL tienes'

'No answer'

,job_company,questions,answers,country,area
0,Analista De Base De Datos at Posadas was asked...,¿Que posición tomarías en caso de que te enter...,Denunciar inmediatamente el mal uso de dicha i...,Mexico,Data Analyst
1,Senior Data Analyst at Accenture was asked...,"Situational behavior, what would you do if som...",They expect you to help them but only if you a...,Mexico,Data Analyst
2,Senior Data Analyst at Accenture was asked...,SQL and ETL questions and answers (imaginary c...,You need to explain step by step the code you ...,Mexico,Data Analyst
3,Data Analyst at Grupo Tress was asked...,Whats your experience? Have you ever worked be...,"You only have to check the company's website, ...",Mexico,Data Analyst
4,analista de datos at Taller de Tornos was aske...,desde conocimiento de herramientas hasta conoc...,sin lugar a duda el trayecto de cada expericienca,Mexico,Data Analyst
5,Senior Data Scientist at Kueski was asked...,Technical and book concepts about learning mod...,Just read the documentation.,Mexico,Data Analyst
6,Data Science at Zigo Capital was asked...,How would you improve the dashboard?,suggested a few kpi's and visuals,Mexico,Data Analyst
7,Científico De Datos at Albo (Mexico) was asked...,Preguntas básicas de clasificación y regresión...,Vale la pena repasar algunos conceptos básico ...,Mexico,Data Analyst
8,Data Analyst at BNN Mexico was asked...,Por que te gustaría desempeñarte en este puesto,Por que es un área en la que me considero fuer...,Mexico,Data Analyst
9,analista de datos at Rosen was asked...,Como me veo en 10 años,Con familia y un buen cargo,Mexico,Data Analyst


In [19]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'Mexico' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Desarrollador Web at Exosfera was asked...'

'Experiencia'

'Nula en el Hambito laboral'

2

...

39

'UI Web Developer at Globant was asked...'

'How would you design a responsive site?'

'No answer'

,job_company,questions,answers,country,area
0,Desarrollador Web at Exosfera was asked...,Experiencia,Nula en el Hambito laboral,Mexico,Web Developer
1,Desarrollador De Aplicaciones Web at Administr...,Qué tal manejas las bases de datos,Lo manejo en un 60%,Mexico,Web Developer
2,Web Developer at Tecnologías Empresariales Apl...,Menciona 5 fortalezas tuyas,"Valentia, Proactivo",Mexico,Web Developer
3,Web Developer at Inoventrix was asked...,How can we use a foreach in PHP?,With the next line you can use a foreach: fore...,Mexico,Web Developer
4,Desarrollador Web at Wachinango was asked...,Si me acoplaría a nuevos lenguajes de programa...,"Si, es cuestión de sintaxis puesto que la lógi...",Mexico,Web Developer
5,Desarrollador De Aplicaciones Web at Alluxi wa...,Si tenia dominio con algun framework para desa...,En ese entonces tenia conocimiento básico de A...,Mexico,Web Developer
6,Desarrollador Web at Tecnologia Aplicada a Neg...,El cliente me pregunto que guardo en mis bolsi...,"mi respuesta fue Celular, llaves , cartera y D...",Mexico,Web Developer
7,Desarrollador Web at Universidad Autónoma Chap...,Ninguna,Nada,Mexico,Web Developer
8,Web Developer at Mahisoft was asked...,En la primera entrevista te preguntan tu traye...,Respondí con transparencia.,Mexico,Web Developer
9,Web Developer at Knotion was asked...,¿Qué puedes aportar a Knotion?,"Conocimiento, capacidad, entusiasmo, innovació...",Mexico,Web Developer


In [20]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'Mexico' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'UX-UI Designer at Accenture was asked...'

'Create a digital solution that help rural communities get access to healthcare. Present at least two screens.'

'I scouted medical apps and analyze them, created a branding identity with proper image argumentation, built a solution. Benefits, partnership, stakeholders and how it worked was defined.'

2

...

5

'UI Designer at Lenovo was asked...'

'What is your previous professional experience?'

'No answer'

,job_company,questions,answers,country,area
0,UX-UI Designer at Accenture was asked...,Create a digital solution that help rural comm...,"I scouted medical apps and analyze them, creat...",Mexico,UI Designer
1,UI/User Experience Designer at Bhuvi IT Soluti...,What is your design process?,easy.,Mexico,UI Designer
2,UX-UI Designer at Oracle was asked...,Cual era mi plan de carrera,No answer,Mexico,UI Designer
3,UI/UX Designer at EasyBroker was asked...,Examples of team work.,No answer,Mexico,UI Designer
4,UI Designer at Lenovo was asked...,What is your previous professional experience?,No answer,Mexico,UI Designer


In [21]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'Italy' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

1

'Data Analyst at Minsait was asked...'

'Tecniche su competenze acquisite'

'Con le mie competenze e conoscenze'

2

...

107

'Data Scientist at Capgemini was asked...'

'What role do you want to cover inside the company'

'No answer'

,job_company,questions,answers,country,area
0,Data Analyst at Minsait was asked...,Tecniche su competenze acquisite,Con le mie competenze e conoscenze,Italy,Data Analyst
1,Data Analyst at Hera was asked...,dove ti vedi tra 10 anni?,ruolo di innovation,Italy,Data Analyst
2,Corporate Events Data Analyst at Bloomberg L.P...,"The psychological test was easy and intuitive,...","I did well on the psychological test, but I th...",Italy,Data Analyst
3,Junior Data Scientist at Damart was asked...,Conosci dei linguaggi in particolare?,"Conosco Java, Python e R",Italy,Data Analyst
4,Data Scientist at Featurespace was asked...,"coding questions: sorting, searching, some dat...",the questions weren't too difficult and the in...,Italy,Data Analyst
...,...,...,...,...,...
102,Data Analyst at Criteo was asked...,Why do you want to work with us?,No answer,Italy,Data Analyst
103,Data Analyst at Accenture was asked...,Questions about my computer science skills and...,No answer,Italy,Data Analyst
104,Data Scientist at Sorgenia was asked...,"sai qualche linguaggi programmazione, tipo pyt...",No answer,Italy,Data Analyst
105,Data Scientist at Capgemini was asked...,Percorso di studi Motivazioni Cosa mi piace de...,No answer,Italy,Data Analyst


In [ ]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'Italy' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'Italy' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'Sweden' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'Sweden' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'Sweden' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'France' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'France' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'France' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'Netherlands' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'Netherlands' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'Netherlands' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'Brazil' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'Brazil' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'Brazil' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'United Kingdom' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'United Kingdom' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [ ]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'United Kingdom' ## Lets search Data analytics

data_to_csv(location_pt,interview_info(job_title_da,location_pt))

In [51]:
driver.close()